We want to parse the \_sent and sent folders and parse their respective dates.

__We will ignore all non-compliant characters from the mails!__

In [48]:
import pandas as pd
import numpy as np
import re
import os

tmp = []
sent_mail_folder_names = ["sent_items", "_sent_mail", "_sent", "sent"]
date_field_regex = re.compile(r"^Date:")
result_dict = {}

def read_maildir(path):
    people = os.listdir(path)
    for person in people:
        current_person_dict = {}
        personal_mailbox_path = os.path.join(path, person)
        folders = os.listdir(personal_mailbox_path)
        merged_folders = [value for value in folders if value in sent_mail_folder_names]
        for folder in merged_folders:
            current_folder_path = os.path.join(personal_mailbox_path, folder)
            current_person_dict.update(read_folder(current_folder_path, person))
        result_dict.update(current_person_dict)
        
def read_folder(path, person):
    result = {}
    children = os.listdir(path)
    for child in children:
        child_path = os.path.join(path, child)
        if os.path.isdir(child_path):
            result.update(read_folder(child_path, person))
        else:
            with open(child_path, "r", encoding="utf8", errors='ignore') as f:
                lines = f.readlines()
                matched_lines = list(filter(date_field_regex.search, lines))
                if len(matched_lines) != 0:
                    result[child_path] = {"name": person, "date": matched_lines[0]}
    return result
   
    
read_maildir("../data/maildir")

In [49]:
df = pd.DataFrame(result_dict).transpose()
df.head()

,name,date
../data/maildir/mckay-b/_sent_mail/44.,mckay-b,"Date: Mon, 7 May 2001 04:44:00 -0700 (PDT)\n"
../data/maildir/mckay-b/_sent_mail/11.,mckay-b,"Date: Tue, 3 Oct 2000 01:18:00 -0700 (PDT)\n"
../data/maildir/mckay-b/_sent_mail/38.,mckay-b,"Date: Tue, 8 May 2001 04:11:00 -0700 (PDT)\n"
../data/maildir/mckay-b/_sent_mail/13.,mckay-b,"Date: Wed, 27 Sep 2000 04:51:00 -0700 (PDT)\n"
../data/maildir/mckay-b/_sent_mail/33.,mckay-b,"Date: Wed, 9 May 2001 09:14:00 -0700 (PDT)\n"


In [50]:
import dateutil.parser as parser
def parse_date(payload):
    return parser.parse(payload.strip()[5:].strip())
    
df["parsed_date"] = df["date"].apply(parse_date)

In [51]:
df["time"] = df["parsed_date"].apply(lambda x: x.time())
df["day"] = df["parsed_date"].apply(lambda x: x.date())
df["tz"] = df["parsed_date"].apply(lambda x: x.tzname())
df.head()

,name,date,parsed_date,time,day,tz
../data/maildir/mckay-b/_sent_mail/44.,mckay-b,"Date: Mon, 7 May 2001 04:44:00 -0700 (PDT)\n",2001-05-07 04:44:00-07:00,04:44:00,2001-05-07,PDT
../data/maildir/mckay-b/_sent_mail/11.,mckay-b,"Date: Tue, 3 Oct 2000 01:18:00 -0700 (PDT)\n",2000-10-03 01:18:00-07:00,01:18:00,2000-10-03,PDT
../data/maildir/mckay-b/_sent_mail/38.,mckay-b,"Date: Tue, 8 May 2001 04:11:00 -0700 (PDT)\n",2001-05-08 04:11:00-07:00,04:11:00,2001-05-08,PDT
../data/maildir/mckay-b/_sent_mail/13.,mckay-b,"Date: Wed, 27 Sep 2000 04:51:00 -0700 (PDT)\n",2000-09-27 04:51:00-07:00,04:51:00,2000-09-27,PDT
../data/maildir/mckay-b/_sent_mail/33.,mckay-b,"Date: Wed, 9 May 2001 09:14:00 -0700 (PDT)\n",2001-05-09 09:14:00-07:00,09:14:00,2001-05-09,PDT
